<a href="https://colab.research.google.com/github/Saba-a11/Crypto/blob/main/Market_Efficiently_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Base Standard Test for Randomness Based on Linearity**

The Base Standard Test for Randomness Based on Linearity (BSTRL) is a statistical test used to measure the randomness of a sequence of numbers. It is based on the concept of linearity, which is the degree to which a sequence of numbers follows a straight line. The BSTRL is used to evaluate the randomness of a sequence of numbers by examining the linearity of the sequence. The test is based on the assumption that a random sequence of numbers should be approximately linear. If the linearity of the sequence is significantly different from a straight line, then the sequence is considered to be non-random.

**The Hurst Exponent Test**

Hurst is a statistical measure that is used to quantify the long-term memory of a time series. It is calculated by analyzing the autocorrelations of the series over different time lags. 
The Hurst Exponent used to quantify the degree of autocorrelation in a series, which is a measure of how well past values of a series can be used to predict future values. 
The Hurst Exponent ranges from 0 to 1, with values closer to 1 indicating more persistent behavior and values closer to 0 indicating less persistent behavior.
A higher Hurst exponent indicates that the series has a greater tendency to return to its previous values, while a lower exponent indicates that the series is more likely to drift away from its previous values.



*   If Hurst = 0.5, then the market is random.
*   If Hurst > 0.5, then there is evidence of a trending market.
*   If Hurst < 0.5, then there is evidence of a mean reverting market.

**The AD Fuller Test**

The Augmented Dickey-Fuller (ADF) test is a statistical test used to determine whether a time series is stationary or non-stationary. It is used to test the null hypothesis that a unit root is present in a time series sample. The ADF test is one of the most widely used tests for determining stationarity in a time series. The test results provide evidence either for or against the presence of a unit root in the series, which is a necessary condition for stationarity.

**The Lyapunov Exponent Test**

The Lyapunov Exponent Test is a mathematical tool used to measure the sensitivity of a system to its initial conditions. This test is used in chaos theory to measure the rate at which a system moves away from its initial conditions over time. The Lyapunov Exponent Test is used to determine whether a system is chaotic or not. It is calculated by taking the natural logarithm of the ratio of two nearby points in the system’s state space and then averaging the logarithms for all points in the system’s state space. The Lyapunov Exponent Test can be used to detect chaotic behavior in systems such as weather patterns, population growth, and stock market movements.

Testing for chaos and nonlinearity. Considered as your last line of defence as takes into account non-linear dependancies after running other efficiency tests.

In [1]:
import numpy as np
import statsmodels.api as sm
import scipy.stats as sps

def dynamic_hurst_component(returns):
    power = 10
    n = 2**power
    
    hursts = np.array([])
    tstats = np.array([])
    pvalues = np.array([])
    res = np.array([])
    
    for t in np.arange(n, len(returns) + 1):
        # Specify subsample
        data = returns[t-n:t]
        X = np.arange(2, power + 1)
        Y = np.array([])
        for p in X:
            m = 2**p
            s = 2**(power-p)
            rs_array = np.array([])
            
            # Move across subsamples
            for i in np.arange(0, s):
                subsample = data[i*m:(i+1)*m]
                mean = np.average(subsample)
                deviate = np.cumsum(subsample-mean)
                difference = max(deviate) - min(deviate)
                stdev = np.std(subsample)
                rescaled_range = difference / stdev
                rs_array = np.append(rs_array, rescaled_range)
                
            # Calculating the log2 of average rescaled range
            Y = np.append(Y, np.log2(np.average(rs_array)))
        reg = sm.OLS(Y, sm.add_constant(X))
        res = reg.fit()
        hurst = res.params[1]
        tstat = (res.params[1] - 0.5) / res.bse[1]
        pvalue = 2 * (1 - sps.t.cdf(abs(tstat), res.df_resid))
        hursts = np.append(hursts, hurst)
        tstats = np.append(tstats, tstat)
        pvalues = np.append(pvalues, pvalue)
        
    return hursts, tstats, pvalues, n, res

In [2]:
import pandas as pd
import numpy as np
from numpy import cumsum, log, polyfit, sqrt, std, subtract
import sklearn.mixture as mix
import yfinance as yf

In [3]:
# Data Extraction
start_date = "2019-12-31"
end_date = "2022-06-30"
symbol = "SPY"
df = yf.download(symbol, start_date, end_date)
df["Returns"] = df["Close"] / df["Close"].shift(1) - 1
df.dropna(inplace=True)
df.head(3)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Returns
Date,,,,,,,
2020-01-02,323.540009,324.890015,322.529999,324.869995,309.694855,59151200,0.009352
2020-01-03,321.160004,323.640015,321.100006,322.410004,307.349854,77709700,-0.007572
2020-01-06,320.489990,323.730011,320.359985,323.640015,308.522369,55653900,0.003815


In [4]:
# Returns extraction
returns = df["Returns"].values.astype(float)

In [5]:
result = dynamic_hurst_component(returns)